In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
#%run -i utils.py
#import seaborn as sns
#import datetime 

## Ingest the data

In [4]:
train_2016 = pd.read_csv('../data/train_2016_v2.csv')
train_2017 = pd.read_csv('../data/train_2017.csv')
train_2017.head()

In [15]:
df_2016 = pd.read_csv('../data/properties_2016.csv')
df_2016.shape

(2985217, 58)

In [14]:
df_2017 = pd.read_csv('../data/properties_2017.csv')
df_2017.shape

/Users/sam_stone/venvs/zillow_prize/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2985217, 58)

In [7]:
df_2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [ ]:
ct_2017 = len(df_2017)
completeness = pd.DataFrame(df_2017.columns, columns=['col_name'])
completeness['type'] = completeness.apply(lambda r: df_2017[r.col_name].dtypes, axis=1)
completeness['%_null'] = completeness.apply(lambda r: df_2017[r.col_name].isnull().sum() / ct_2017, axis=1)

In [123]:
completeness.sort_values('%_null')

,col_name,type,%_null
0,parcelid,int64,0.000000
17,fips,float64,0.000982
33,propertylandusetypeid,float64,0.000982
35,rawcensustractandblock,float64,0.000982
37,regionidcounty,float64,0.000982
25,longitude,float64,0.000982
24,latitude,float64,0.000982
52,assessmentyear,float64,0.000983
5,bedroomcnt,float64,0.000987
4,bathroomcnt,float64,0.000991


In [27]:
# Do 2016 and 2017 parcel_ids match? Yes.
set(df_2016.parcelid) == set(df_2017.parcelid)

True

In [86]:
# Do assessment values differ in 2016 and 2017? Yes, by +1.5% median
df = df_2016.join(df_2017.set_index('parcelid'), on='parcelid', how='left', rsuffix='_2017')

df['delta'] = df['taxvaluedollarcnt_2017'] / df['taxvaluedollarcnt'] - 1.0
df[[
    'parcelid',
    'taxvaluedollarcnt',
    'taxvaluedollarcnt_2017',
    'delta',
]].head(5)
df.delta.median()

0.015247363301987393

In [90]:
# Does total assessment value equal structure + land value? Yes
df['assessment_value_check'] = df.taxvaluedollarcnt - df.landtaxvaluedollarcnt - df.structuretaxvaluedollarcnt
df.assessment_value_check.sum()
#df[['taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'assessment_value_check']]

0.0

### Map Dictionary Values

In [179]:
prop_types = pd.read_excel('../data/zillow_data_dictionary.xlsx', sheet_name='PropertyLandUseTypeID')
prop_types

ImportError: Install xlrd >= 0.9.0 for Excel support

## Predict total assessment value in 2017
- Filter to only legitimate total assessed values ($100K-$1M)
- Filter to only columns that are >90% complete (there is a natural split here)
- Imput missing data for selected columns
- Examine categorical features
- Run regression
- Features to consider adding back
 - FIPS
 - Various location features
 - Land Use Code
 - Bath counts
 

In [147]:
features = [
#     'fips', # Federal Information Processing Standard code
    'propertylandusetypeid', # Categorical - 16 values
#     'rawcensustractandblock',
#     'regionidcounty',
#     'longitude',
#     'latitude',
#     'assessmentyear', #Almost 100% from 2016
    'bedroomcnt', # Exclude count > 6
    'bathroomcnt', # Exclude count > 10
    'roomcnt', # Exclude count > 20
#     'propertycountylandusecode', - Long tail, examine in more detail
#     'regionidzip',
#     'taxamount', #Data leakage?
    'calculatedfinishedsquarefeet', 
    'yearbuilt',
#     'regionidcity',
#     'censustractandblock',
#     'fullbathcnt', #Consider for addback
#     'calculatedbathnbr', #Consider for addback
    'finishedsquarefeet12',
    'lotsizesquarefeet'
]

labels = [
    'taxvaluedollarcnt',
    'structuretaxvaluedollarcnt',
    'landtaxvaluedollarcnt',
]

In [173]:
# Filter on total assessed value
df = df_2017[features + labels]
df = df[(df.taxvaluedollarcnt > 100000) & (df.taxvaluedollarcnt < 1000000)]
df.describe()
df.shape

(2430767, 11)

In [174]:
# Filter on bedroom, bathroom, and total room counts
df = df[df.bedroomcnt <= 6]
df = df[df.bathroomcnt <= 10]
df = df[df.roomcnt <= 20]
df.shape

(2403675, 11)

In [175]:
#Drop all rows with null values
tmp = df.dropna()
print('{:.2f}% dropped'.format((1 - len(tmp) / len(df)) * 100))
df = tmp
df.shape

14.64% dropped


(2051864, 11)

In [176]:
df.isnull().sum()

propertylandusetypeid           0
bedroomcnt                      0
bathroomcnt                     0
roomcnt                         0
calculatedfinishedsquarefeet    0
yearbuilt                       0
finishedsquarefeet12            0
lotsizesquarefeet               0
taxvaluedollarcnt               0
structuretaxvaluedollarcnt      0
landtaxvaluedollarcnt           0
dtype: int64

In [177]:
df.propertylandusetypeid.value_counts()

261.0    1718946
266.0     266573
269.0      51644
265.0       8235
246.0       1842
263.0       1573
267.0       1272
248.0        565
247.0        461
260.0        315
275.0        312
264.0        126
Name: propertylandusetypeid, dtype: int64